## Breast Cancer: Causes and Drug Target Prediction

The program analyzes the cause of breast cancer from a Boolean network datamodel, namely which genes should be mutated to cause cancer and are they oncogenes (True) or tumor suppressors (False).
Next, the program found the potential therapeutic targets and the actions on them for a cancer caused by BRCA1 mutation.

In [ ]:
# BREAST CANCER CAUSES AND DRUG PREDICTION
import time
import matplotlib.pyplot as plt
from sympy import SOPform, simplify_logic
from tabulate import tabulate
from boon import *

## Network model
import the network model from a text file. Note that this model also exists in BooN format. Note that in a normal cell these two phenotypes exist potentially. 

In [ ]:
#  Load the network model. A text version of the model in BNet is also available
bcn = BooN.load("breast-cancer")

# Show the Boolean network
print(" - Boolean Network - ")
print(bcn)

In [ ]:
# print all the genes.
print("\n", "List of genes: ", bcn.variables)

### Stable States
In this model two stable states exist that respectively correspond to cell division and apoptosis. 

In [ ]:
# Compute the stable states.
print("\n" * 2, " - Stable States -")
stable_states = bcn.stable_states
print(tabulate(stable_states, headers='keys', tablefmt='plain'))

### Interaction Graph

In [ ]:
print("Interaction Graph")
# interaction graph
bcn.draw_IG()
plt.show()  # Show the interaction graph

## Cause of Cancer
In order to ascertain the potential causes of cancer, a controllability analysis is first employed.
This analysis is based on the characterisation of biomarkers, which are genes whose states are utilised to differentiate between phenotypes.
Based on the states of the aforementioned biomarkers, a control query is then defined with the objective of identifying the cause of cancer. 

### Characterization of the biomarkers
The selected biomarkers are CycD1 and Bax. The Boolean profiles, respectively, correspond to cell division and apoptosis.


| CyCD1 | Bax | *Phenotype* |
|:-----:|:-----:| -------:|
| 0 | 1 | *Apoptosis* |
| 1 | 0 | *Cell Division* |


In [ ]:
# Define the biomarkers
CycD1, Bax = symbols("CycD1 Bax")
biomarkers = {CycD1, Bax}

In essence, the model represents the cancerous state as a loss of apoptosis, which is a hallmark of cancer.
We will define a query, or formula, that delineates the circumstances under which apoptosis cannot be sustained in any stable state. 

In [ ]:
# Characterize a marking corresponding to the apoptosis
marking = {CycD1: False, Bax: True}
# convert the marking into a formula
apoptosis_marking = SOPform(biomarkers, [marking])
print("Apoptosis biomarkers")
print(tabulate([marking], headers='keys', tablefmt='plain'))


### Control query definition
Set the marking formula of cancer loosing the apoptosis, that is the negation of apoptosis query, A cell therefore cannot die defining a hallmark of cancer.

In [ ]:
 kquery = simplify_logic(~apoptosis_marking, force=True)
print("Disease query, prevent apoptosis: ", kquery)

Now, we Define the variables where the control can be applied to force the freeze of variables.
A variable can have a two controls for freezing to True and to False.
Obviously, the biomarkers are always excluded from the control since they are observers characterizing the phenotypes by their profile. 

In [ ]:
frozenvars0 = frozenvars1 = bcn.variables - biomarkers
print("Frozen variables 0 and 1")
print(tabulate([frozenvars0, frozenvars1], tablefmt='grid'))

Finally, we extend the Boolean network to a Boolean Controlled network according to the frozen variables. 

In [ ]:
# Copy the network and fix the control on it. The control consists in adding control parameters (_u0<var>, _u1<var>) to formulas. 
bcc = bcn.copy()
bcc.control(frozenvars0, frozenvars1)

print("\n- Controlled Network -")
print(bcc)

 ### Controllability Analysis
A controllability analysis should be conducted for  identifying  the potential gene freezes that would satisfy the query at stable state. 

The number of models is approximately 3,500. We kindly request your patience.

In [ ]:
print("\nQuery: Necessary avoid the apoptosis.")
start_time = time.time()
destiny = bcc.necessary(kquery, trace=True)  # activate the trace to see the evolution of the computation.
print("Runtime..: {:5.2f}".format(time.time() - start_time))
print("# clauses: {}".format(len(destiny.args)))
print("# symbols: {}".format(destiny.count(Symbol)))

### Controllability Analysis

In [ ]:
print("\nCore")
print("About 10 solutions.")
# Compute the core.
start_time = time.time()
core = BooN.destify(destiny, trace=True)
print("Runtime: {:5.2f}".format(time.time() - start_time))

From the core, we derive the potential freeze actions as a pair (gene, freeze action).
The table comprises a series of lines, each corresponding to a specific freeze action on genes that is necessary to fulfil the query.
In the event that two pairs are present within a single line, it is necessary to combine both actions. 

In [ ]:
print("\nActions")  # Transform the core into actions.
actions = core2actions(core)
print(tabulate(actions))

## Drug Target identification
We now identify the possible drug target associated with the expected action to cure BRCA1 Cancer.

In [ ]:
# We define a situation of cancer with BRCA1 mutation.
print("\n" * 2, " - Mutate BRCA1  -")
BRCA1 = symbols("BRCA1")
bcn.desc[BRCA1] = False

print("Mutated network.")
print(bcn)

Define the frozen variables from the previous situation of the frozen variables by removing BRCA1

In [ ]:
# copy the network and apply control
# Remove the mutated gene from the controlled variable since no modifications is possible.
frozenvars0 = frozenvars1 = frozenvars0 - {BRCA1}
bndrug = bcn.copy()  # fix control on a new instance of bcn.
bndrug.control(frozenvars0, frozenvars1)

### Controllability analysis
The issue is to identify the actions that induce apoptosis in order to determine the most effective therapeutic targets.
It is hypothesised that at least a stable state will fulfil the query property since the drug may not work for all people. 

Note that the possibility resolution is faster than the necessity.

In [ ]:

print("\nQuery : Possibly lead to apoptosis.")
start_time = time.time()
destiny = bndrug.possibly(apoptosis_marking)
print("Runtime..: {:5.2f}".format(time.time() - start_time))
print("# clauses: {}".format(len(destiny.args)))
print("# symbols: {}".format(destiny.count(Symbol)))

In [ ]:
print("\nCore")
start_time = time.time()
core = BooN.destify(destiny)
print("Runtime: {:5.2f}".format(time.time() - start_time))

From the core, we finally derive the potential freeze actions as a pair (gene, freeze action) to gain the apoptosis.

In [ ]:
print("\nActions")  # Transform the core into actions.
actions = core2actions(core)
print(tabulate(actions))